### Histórico de Versões
V1 - 01/06/2020

# Contexto

Esse é um projeto de machine learning desenvolvido em dupla, sendo o primeiro projeto de ambos os membros. Com o objetivo de aprender sobre data science e machine learning, resolvemos nos unir e trabalhar em uma das competições propostas pela Kaggle.


#### Sobre o naufrágio do Titanic

O RMS Titanic, ou só Titanic, como é popularmente conhecido, foi um navio de passageiros de origem britânica. Tido como "inafundável", foi projetado para ser o navio mais seguro de sua época, o que pode ter gerado demasiada segurança por parte de sua tripulação e piloto, uma vez que, em sua viagem inalgural, o Titanic encontrou seu trágico fim.
Na noite de 14 de abril de 1912, após se chocar com um iceberg, iniciou-se uma das maiores tragédias conhecidas popularmente, o naufrágio do titanic. 
Parte do que compôs sua tragédia foi a ausência de botes salva-vidas o suficiente para as pessoas a bordo (um total de 2208), o que causou indignação nos sobreviventes, e foi o estopim para a criação, em 1914, da Convenção Internacional para a Salvaguarda da Vida Humana no Mar(SOLAS, em inglês).
Atualmente, o Titanic encontra-se no fundo do Atlântico Norte, em constante corrosão causada pelo sal presente na água.

# Machine Learning

Como dito anteriormente, somos 2 estudantes buscando aprofundar nossos conhecimentos em machine learning e data science.
Sendo assim, vamos a parte que interessa

### Importando as bibliotecas

Iniciamos com a importação das bibliotecas que serão utilizadas no projeto de machine learning

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math 
import seaborn as sns
import missingno
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import itertools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Importando os dados

Aqui importamos as bases de dados que serão utilizadas para treinar e testar o modelo, respectivamente

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

### Averiguando os dados

Aqui optamos por expor algumas das informações sobre as bases de dados, para que seja mais fácil de se ter uma ideia de com o que estamos trabalhando

In [ ]:
train.head(30)

In [ ]:
train.describe()


In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
print("A base de treino possui:")
print(train.memory_usage().sum(),"Bytes ou ")
print(train.memory_usage().sum()/1024,"KBytes ou ")
print(train.memory_usage().sum()/1024/1024,"MBytes")

In [ ]:
print("E a base de teste... :")
print(test.memory_usage().sum(),"Bytes ou ")
print(test.memory_usage().sum()/1024,"KBytes ou ")
print(test.memory_usage().sum()/1024/1024,"MBytes")

In [ ]:
print("Informações acerca de Train")
print("_"*40)
train.info()

In [ ]:
print("Informações acerca de Test")
print("_"*40)
test.info()

In [ ]:
print("Dimensões de Train")
print("Linhas, Colunas")
print(train.shape)

In [ ]:
print("Dimensões de Test")
print("Linhas, Colunas")
print(test.shape)

In [ ]:
print("Essas são todas as variáveis presentes: ", list(train))
print("_"*142)
print("|"*142)
print("_"*142)
print("Feita uma pequena análise, percebe-se que algumas são mais promissoras que outras, então "
      +"separamos algumas para verificar possiveis conjutos")

cont = 0
while cont<8:
    cont = cont+1
    for i in itertools.combinations(list(train[["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]]),cont):
        print(i)

# Observações e Teorias

-Provavelmente mulheres e crianças foram evacuadas do navio antes dos homens, o que sugere que a variável de idade (<b>Age</b>) e de sexo (<b>Sex</b>) podem ser importantes

-Um dos motivos que gerou indignação após o naufrágio do Titanic foi o descaso para com as classes socioeconômicas inferiores, o que sugere que mais indivíduos das classes altas sobreviveram, tornando <b>Pclass</b> uma variável interessante

-Existe uma boa chance de indivíduos que embarcaram em localidades diferentes terem sido alocados em cabines em regiões diferentes, o que sugere que talvez algum dos locais de embarque estivesse mais próximo de botes ou em uma região que demorou para afundar ou quebrar no navio, atribuindo assim valor a variavel <b>Embarked</b>

-Possuir pais ou filhos presentes também no navio pode implicar em a pessoa ser criança ou estar acompanhada de crianças. Também é possivel que, indivíduos sem dados de idade mas com dados de familia e sobrevivencia tem maior chance de serem crianças

# Definindo Variáveis
<body style="background-color:blue">
Após uma breve análise, escolhemos algumas variáveis que apresentam maior potencial<br>
-Sex<br>
-Age<br>
-Embarked<br>
<br>
<br>
É possível perceber que a coluna Age apresenta campos vazios, sendo assim, vamos fazer um pequeno teste, imputando o valor mediano nos campos vazios
</body>

In [ ]:

variaveis = ["SexBinario", "SibSpSex","Pclass", "EmbarkedNumerico"]

### Criando novas colunas

Agora que criamos as funções capazes de transformar os campos Sex em binário e os campos Embarked em numérico, vamos adicionar 2 colunas, com os valores transformados, nas tabelas de test e train

In [ ]:
def SexBinario(valor):
    if valor == "female":
        return 1
    else:
        return 0
    
def EmbarkedNumerico(valor):
    if valor == "S":
        return 1
    elif valor == "C":
        return 2
    else:
        return 3
    
def GrupoIdade(idade):
    if idade>=65 :
        return 3
    elif idade < 65 and idade >=18 :
        return 2
    else:
        return 1

def SibSpBinario(sibsp):
    if sibsp != 1:
        return 0
    else:
        return 1
    
def CategFare(valor):
    if valor <= 100:
        return 0
    elif valor >100:
        return 1
    
def AgeBinario(valor):
    if valor<7:
        return 1
    else:
        return 0
    
def MenorDe7Anos(valor):
    if valor<7:
        return 1
    else:
        return 0
def SibSpSex(c):
    if c.Sex == "female" and c.SibSp <4:
        return 1
    else:
        return 0
    


In [ ]:
train["EmbarkedNumerico"] = train["Embarked"].map(EmbarkedNumerico)
train["SexBinario"] = train["Sex"].map(SexBinario)
train["GrupoAge"] = train["Age"].map(GrupoIdade)
train["SibSpBinario"] = train["SibSp"].map(SibSpBinario)
train["CategFare"] = train["Fare"].map(CategFare)
train["AgeBinario"] = train["Age"].map(AgeBinario)
train["MenorDe7Anos"] = train["Age"].map(MenorDe7Anos)
train["SibSpSex"] = train.apply(SibSpSex, axis=1)


vivos = train[train["Survived"] == 1]


test["EmbarkedNumerico"] = test["Embarked"].map(EmbarkedNumerico)
test["SexBinario"] = test["Sex"].map(SexBinario)
test["GrupoAge"] = test["Age"].map(GrupoIdade)
test["SibSpBinario"] = test["SibSp"].map(SibSpBinario)
test["CategFare"] = test["Fare"].map(CategFare)
test["AgeBinario"] = test["Age"].map(AgeBinario)
test["MenorDe7Anos"] = test["Age"].map(MenorDe7Anos)
test["SibSpSex"] = test.apply(SibSpSex, axis=1)
vivos

Porém, para utilizar-mos a RandomForest, precisamos de dados numéricos, e, como é possível de se perceber, Sex e Embarked são campos textuais. Sendo assim, vamos criar 2 funções que visam transforma-los em números. Note que, isso só é recomendado porque não existem campos vazios na coluna "Sex" e nem na coluna "Embarked"<br>
<br>
Para a coluna "Age", vamos fazer um pequeno teste, imputando o valor mediano nos campos vazios

# Gráficos - Homens

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "male"], hue="Survived",y="Pclass")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "male"], hue="Survived",y="Sex")

In [ ]:
plt.figure(figsize=(16,16))
sns.countplot(data=train[train["Sex"] == "male"], hue="Survived",y="Age")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "male"], hue="Survived",y="SibSp")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "male"], hue="Survived",y="Parch")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "male"], hue="Survived",y="CategFare")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "male"], hue="Survived",y="EmbarkedNumerico")

# Gráficos - Mulheres

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "female"], hue="Survived",y="Pclass")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "female"], hue="Survived",y="Sex")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "female"], hue="Survived",y="Age")

In [ ]:
plt.figure(figsize=(16,8))
sns.countplot(data=train[train["Sex"] == "female"], hue="Survived",y="SibSp")

# Graficos - Geral

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Sobrevivencia por SibSpSex")
sns.countplot(data=train, hue="Survived",y="SibSpSex")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de Idade por Local de Embarque")
sns.countplot(data=train, hue="GrupoAge",y="Embarked")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de Survived por Pclass")
sns.countplot(data=train, hue="Survived",y="Pclass")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de Survived por Embarked")
sns.countplot(data=train, hue="Survived",y="Embarked")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de idade por Survived")
sns.countplot(data=train, hue="Survived",y="GrupoAge")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de Parch por Survived")
sns.countplot(data=train, hue="Survived",y="Parch")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de SibSp por Survived")
sns.countplot(data=train, hue="Survived",y="SibSp")

In [ ]:
plt.figure(figsize=(16,16))
plt.title("Idade por Survived")
sns.countplot(data=train, hue="Survived",y="Age")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de Survived por Sex")
sns.countplot(data=train, hue="Survived",y="Sex")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de Survived por Embarked")
sns.countplot(data=train, hue="Survived",y="Pclass")

In [ ]:
plt.figure(figsize=(16,8))
plt.title("Grupo de Survived por CategFare")
sns.countplot(data=train, hue="Survived",y="CategFare")

# Definindo modelo e variáveis de treino

Agora que temos as colunas com as variáveis desejadas, precisamos organiza-las em 2 variáveis, sendo uma delas para o treinamento, e a outra com as respostas desse treinamento. Dessa forma, o modelo que iremos criar terá uma base de como deve se parecer certos dados para que eles resultem em certas respostas

In [ ]:
X = train[variaveis]
y = train["Survived"]
X_prev = test[variaveis]

Como convenção, utilizamos a variável X para designar os dados para o calculo, y para designar as respostas corretas para esses calculos. X_prev age como a variável dos dados do arquivo de teste, o qual vamos ter que descobrir, por meio do machine learning, quais os resultados corretos.
Vale ressaltar também que, algumas entradas relacionadas a idade (age) ainda precisam ser tratadas, uma vez que apresentam campos NaN. Faremos isso a seguir

In [ ]:
X = X.fillna(-1)
X_prev = X_prev.fillna(-1)

E por fim, instanciamos o modelo. RandomForestClassifier é o objeto que estamos utilizando para fazer nossa predição.

In [ ]:
titanic_model = RandomForestClassifier(n_estimators=100,n_jobs=1,random_state=0)

# Treinando o Modelo

Agora finalmente chegou a hora de treinar o modelo, em breve teremos um csv "quentinho" pra submissão

In [ ]:
titanic_model.fit(X,y)

Modelo treinado!<br>
Vamos agora predizer X_prev

In [ ]:
p = titanic_model.predict(X_prev)

In [ ]:
p

O gigantesco array visto ai encima é o resultado da predição. Para submeter ao kaggle, precisamos formata-lo, de modo que o mesmo faça parte de uma coluna, chamada de "Survived", juntamente com o "PassengerId" como index

In [ ]:
sub = pd.Series(p, index=test['PassengerId'],name="Survived" )
sub.to_csv("/kaggle/working/modelo_titanic_30.csv")